In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
data=pd.read_csv(r"C:\Users\mail2\OneDrive\Desktop\Ml_Data\NLP_DATA\medicine.csv")

In [3]:
data.head(1)

,index,Drug_Name,Reason,Description
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,Acne,Mild to moderate acne (spots)


In [4]:
#data.isnull().sum()

#data.duplicated().sum()

### Description

In [5]:
data["Description"][0]

'Mild to moderate acne (spots)'

In [6]:
data["Description"][0].split(" ")

['Mild', 'to', 'moderate', 'acne', '(spots)']

In [7]:
#split into token 
#data["Description"].apply(lambda x:x.split(" "))

In [8]:
data["Description"]=data["Description"].apply(lambda x:x.split(" "))
data["Reason"]=data["Reason"].apply(lambda x:x.split(" "))


In [9]:
data.head(1)

,index,Drug_Name,Reason,Description
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,[Acne],"[Mild, to, moderate, acne, (spots)]"


In [10]:
data['Description'] = data['Description'].apply(lambda x:[i.replace(" ","") for i in x])

In [11]:
data['Description'] = data['Description'].apply(lambda x:[i.replace(" ","") for i in x])

In [12]:
data.head(1)

,index,Drug_Name,Reason,Description
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,[Acne],"[Mild, to, moderate, acne, (spots)]"


In [13]:
#comining drug_name and Description

In [14]:
data["Reason_Description"]=data["Description"]+data["Reason"]

In [15]:
data.head(1)

,index,Drug_Name,Reason,Description,Reason_Description
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,[Acne],"[Mild, to, moderate, acne, (spots)]","[Mild, to, moderate, acne, (spots), Acne]"


In [16]:
recom_df=data[["index","Drug_Name","Reason_Description"]]

In [17]:
recom_df.head(1)

,index,Drug_Name,Reason_Description
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,"[Mild, to, moderate, acne, (spots), Acne]"


In [18]:
recom_df["Reason_Description"]=recom_df["Reason_Description"].apply(lambda x:" ".join(x))

In [19]:
recom_df.head(1)

,index,Drug_Name,Reason_Description
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,Mild to moderate acne (spots) Acne


In [20]:
recom_df["Reason_Description"]=recom_df["Reason_Description"].apply(lambda x:x.lower())

In [21]:
recom_df.head(1)

,index,Drug_Name,Reason_Description
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,mild to moderate acne (spots) acne


In [22]:
import nltk
from nltk.stem.porter import PorterStemmer

In [23]:
stem_ps=PorterStemmer()

In [27]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [28]:
def porter_step(text):
    list_y=[]
    for i in text.split():
        list_y.append(stem_ps.stem(i))
    return " ".join(list_y)

In [29]:
recom_df["Reason_Description"]=recom_df["Reason_Description"].apply(porter_step)

In [30]:
  recom_df.head(1)

,index,Drug_Name,Reason_Description
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,mild to moder acn (spots) acn


In [35]:
tf_idf_vector=TfidfVectorizer(use_idf=True,ngram_range=(2,2),stop_words="english")
#vector=tf_idf_vector

In [36]:
vector_tfid=tf_idf_vector.fit_transform(recom_df["Reason_Description"]).toarray()

In [38]:
#tf_idf_vector.get_feature_names()

In [39]:
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
vector_similarity=cosine_similarity(vector_tfid)

In [42]:
#vector_similarity

In [43]:
#recommnedation fun

In [58]:
def medicine_recommendation(medicine):
    index_med = recom_df[recom_df['Drug_Name'] == medicine].index[0]
    distances = vector_similarity[index_med]
    recomm_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in recomm_list:
        print(recom_df.iloc[i[0]].Drug_Name)
    

In [63]:
recom_df.Drug_Name[1]

'A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA Ret 0.025% Gel 20gm'

In [68]:
medicine_recommendation("Acne UV SPF 30 Gel 30gm")

Acnesol 1% Solution 25mlAcnesol Gel 20gmAcnesol Solution 45ml
Acnil Soap 75gm
Adacin Gel 15gm
Airol Cream 20gmAirol 0.03% Cream 20gmAirol 0.05% Cream 20gm
AP Gel 10gm


In [67]:
data[data["Drug_Name"]=="Acnesol 1% Solution 25mlAcnesol Gel 20gmAcnesol Solution 45ml"]["Reason"]

20    [Acne]
Name: Reason, dtype: object